In [3]:
"""
RAG 구성이해및활용 : RAG의구성요소및주요흐름에대해이해하고,그활용방법을알고있나
관련Library 활용 : llama-index, mcp등의RAG 관련library들을적절하게사용할수있나
"""

'\nRAG 구성이해및활용 : RAG의구성요소및주요흐름에대해이해하고,그활용방법을알고있나\n관련Library 활용 : llama-index, mcp등의RAG 관련library들을적절하게사용할수있나\n'

In [ ]:

def parse_htmls(search_results):
    all_documents = []

    # Process each HTML text from the search results to extract text content.
    for html_text in search_results:

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(html_text["page_result"], features="lxml") 
        # html_text["page_result"]는 HTML 문자열을 담고 있는 변수
        # soup는 HTML 구조를 트리 형태로 변환한 파싱 결과
        text = soup.get_text(" ", strip=True)  # Use space as a separator, strip whitespaces
        all_documents.append(text)

    return all_documents

In [ ]:
def extract_chunks(all_documents):
    # Initialize a list to hold all extracted sentences from the search results.
    all_chunks = []

    for document in all_documents:

        if not document:
            # If no document is extracted, add an empty string as a placeholder.
            all_chunks.append("")
        else:

            # Extract offsets of sentences from the document
            _, offsets = text_to_sentences_and_offsets(document)

            # Initialize a list to store sentences
            chunks = []

            # Iterate through the list of offsets and extract sentences
            for start, end in offsets:
                # Extract the sentence and limit its length
                chunk = document[start:end][:MAX_CONTEXT_SENTENCE_LENGTH]
                # 앞에건 문자열 슬라이싱
                # 뒤에 붙인 [...]를 통해 잘라낸 문자열의 길이를 최대 MAX_CONTEXT_SENTENCE_LENGTH까지만 제한
                all_chunks.append(chunk)

    return all_chunks

In [ ]:
class BaseRetriever:
    def __init__(self,):
        self.client = openai.OpenAI(api_key = os.environ["OPENAI_API_KEY"])

    def embed_text(self, texts):
        """Generate embeddings using OpenAI's embedding model."""
        if isinstance(texts, str):
            texts = [texts]

        response = self.client.embeddings.create(
            model="text-embedding-3-small",
            input=texts
        )

        # Extract embeddings correctly from the response object
        embeddings = [np.array(item.embedding) for item in response.data]  # Adjust based on actual attributes
        return np.array(embeddings)

    def retrieve(self, query, search_results, topk):
        # Get documents
        all_documents = parse_htmls(search_results)

        # Get chunks
        all_chunks = extract_chunks(all_documents)

        # Generate embeddings for all chunks and the query.
        all_embeddings = self.embed_text(all_chunks)
        query_embedding = self.embed_text(query)[0]  # Single query embedding

        # Calculate cosine similarity between query and sentence embeddings, and select the top sentences.
        cosine_scores = np.dot(all_embeddings, query_embedding) / (
            np.linalg.norm(all_embeddings, axis=1) * np.linalg.norm(query_embedding)
        )
        top_k_indices = (-cosine_scores).argsort()[:topk]
        # argsort는 기본적으로 오름차순 정렬. 따라서 앞에 cosine_scores에 음수를 취해준다. 
        top_k_chunks = np.array(all_chunks)[top_k_indices]

        return top_k_chunks

In [ ]:
from llama_index.core.schema import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, Settings
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.embed_model = OpenAIEmbedding(model = "text-embedding-3-small")

class LlamaIndexRetriever:
    def __init__.(self):
        self.parser = SentenceSplitter(chunk_size=512, chunk_ovelap=0)
    def retrieve(self, query, search_results, topk):
        documents = []

        for document in parse_htmls(search_results):
            if not document:
                documents.append(Document(text=""))
            else:
                documents.append(Document(text=document))
        base_index = VectorStoreIndex.from_documents(document=documents, transformation=[self.parser])

        base_retriever = base_index.as_retriever(similarity_topk=topk)

        retrieved_nodes = base_retriever.retrieve(query)

        retrieved_results = [retrieved_node.node.get_count().strip() for retrieved_node in retrieved_nodes]
        #  앞뒤 공백을 제거한 깨끗한 문자열 리스트 형태로 얻는걸 주의해야 할 듯 앞 뒤 [] 
        return retrieved_results


In [ ]:
### 시험에 나오지는 않을거 같음. 별 내용이 있는건 아님....
def prompt_generator(query, top_k_chunks, system_prompt):
    user_message = ""
    references = ""

    if len(top_k_chunks) > 0:
        references += "# References \n"
        # Format the top sentences as references in the model's prompt template.
        for chunk_id, chunk in enumerate(top_k_chunks):
            references += f"- {chunk.strip()}\n"

    references = references[:MAX_CONTEXT_REFERENCES_LENGTH]
    # Limit the length of references to fit the model's input size.

    user_message += f"{references}\n------\n\n"
    user_message += f"Using only the references listed above, answer the following question: \n"
    user_message += f"Question: {query}\n"

    llm_input = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_message},
    ]

    return llm_input

In [ ]:
from openai import OpenAI

oai_client = OpenAI()

class Reader:
  def __init__(self):

    self.system_prompt = """
        잘 검색해줘라 뭐 이런 내용을 블라블라 써 놓음. 자체로 문제는 X
    """

  def generate_response(self, query: str, top_k_chunks: list) -> str:
      """
       prompt generator를 통해 llm_input을 만들고, oai_client에 어떤식으로 전달하는지 정도 기억은 해두는게
      """
      llm_input = self.prompt_generator(query, top_k_chunks)
      completion = oai_client.chat.completions.create(
      model="gpt-3.5-turbo",
      temperature=0,
      messages= llm_input
      ).choices[0].message.content # completion 뒤에 채우라고 나오면 여기가 제일 중요하지 않을까
                                  # [0]이 첫번째로 선택된 답변이라는걸 기억해두기      
      return completion

  def prompt_generator(self, query, top_k_chunks):
      user_message = ""
      references = ""

      if len(top_k_chunks) > 0:
          references += "# References \n"
          # Format the top sentences as references in the model's prompt template.
          for chunk_id, chunk in enumerate(top_k_chunks):
              references += f"- {chunk.strip()}\n"
      
      references = references[:MAX_CONTEXT_REFERENCES_LENGTH]
      # Limit the length of references to fit the model's input size.

      user_message += f"{references}\n------\n\n"
      user_message
      user_message += f"Using only the references listed above, answer the following question: \n"
      user_message += f"Question: {query}\n"

      llm_input = [
        {"role": "system", "content": self.system_prompt},
        {"role": "user", "content": user_message},
      ]

      return llm_input

In [ ]:
"""
    아래 3가지 RAG / RAGWithKG / RAGWithSRKG 
    3가지 방법에서 뭐가 다른지 비교하기 파악은 해두기
"""

In [ ]:
class RAG:
    def __init__(self):
        self.retriever = LlamaIndexRetriever()
        self.reader = Reader()

    def inference(self, query, search_results, topk):
        # 1. 관련된 chunk를 검색하고, retrieve할 때는 topk까
        retrieved_results = self.retriever.retrieve(query, search_results, topk)

        # 2. 답변을 생성하는 순서 기억은 해둬야 할 듯.
        answer = self.reader.generate_response(query, retrieved_results)

        return answer, retrieved_results

rag = RAG()
answer = rag.inference(item['query'], item['search_results'], topk)[0]
# 이런식으로 inference 파라미터로 item['query']처럼 넣어주는 것과 맨 뒤에 [0]이 붙어야 하는거 잊지 않기 (하나만 취할 때)

In [ ]:
class RAGWithKG:
    def __init__(self):
        self.kg_query_engine = KGQueryEngine() # RAG의 Llama를 KGQuery로 바꾼 차이점만
        self.reader = Reader()

    def inference(self, query):
        # 1. retrieve relevant kg results
        kg_results, is_finance = self.kg_query_engine.query(query)

        # 2. answer the question based on the retrieved chunks
        answer = self.reader.generate_response(query, [kg_results]) # 여기 kg_results는 [] 씌워서

        return answer, kg_results

In [8]:
class RAGWithSRKG:
    def __init__(self): # 얘는 Llama와 KGQuery다 사용
        self.retriever = LlamaIndexRetriever()
        self.kg_query_engine = KGQueryEngine(server=server) # server=server는 옵션인듯, 
        self.reader = Reader()

    def inference(self, query, search_results, topk):
        #inference를 하나로 통일할수도 있고, 
        # inference에서 retrieve, generate_response를 따로 호출하게 할 수도 있음. 내용은 동일

        # 1. retrieve relevant chunks
        retrieved_results = self.retriever.retrieve(query, search_results, topk)

        # 2. retrieve relevant kg results
        kg_results, is_finance = self.kg_query_engine.query(query)

        # combined_results = [kg_results]
        # combined_results.extend(retrieved_results)
        # is_finance에 따라 [kg_results]를 쓸건지, retrieved_results를 쓸건지 선택하는 차이점만
        if is_finance:
          combined_results = [kg_results]
        else:
          combined_results = retrieved_results
        ### RAGwithoutSR 에서는 위의 is_finance 관련 코드만 삭제 ###

        # 3. answer the question based on the retrieved chunks
        answer = self.reader.generate_response(query, combined_results)

        return answer, combined_results # 그래서 2번째 return값도 combined~를

In [ ]:
# 여기에서는 굳이 문제나올 부분이 있을까??? 
# 그런데 class KGQueryEngineWithMCP:를 이해하려면 같이 비교해보긴 해야할 듯.
class KGQueryEngine:
    def query(self, query):
        generated_query, is_finance = self.generate_query(query)

        if is_finance:
            kg_results = self.get_finance_kg_results(generated_query)
        else:
            kg_results = ""

        return kg_results

    def generate_query(self, query):
        llm_input = prompt_generator(query)
        completion = oai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=0,
        messages=
        llm_input
        ).choices[0].message.content

        try:
            completion = json.loads(completion)
        except:
            completion = extract_json_objects(completion)

        if "domain" in completion.keys():
            domain = completion["domain"]
            is_finance = domain == "finance"
        else:
            is_finance = False

        return completion, is_finance

    def get_finance_kg_results(self, generated_query):
        formatted_time_list = []
        if 'datetime' in generated_query:
            datetime_list = generated_query['datetime'].split(' - ')
            for datetime in datetime_list:
                formatted_time_list.append(convert_to_standard_format(datetime.strip()))


        kg_results = []
        res = ""
        if "market_identifier" in generated_query.keys() and generated_query["market_identifier"] is not None:
            if isinstance(generated_query["market_identifier"], str):
                company_names = generated_query["market_identifier"].split(",")
            else:
                company_names = generated_query["market_identifier"]

            for company_name in company_names:
                try:
                    res = api.finance_get_company_name(company_name)["result"]

                    if res == []:
                        ticker_name = company_name.upper()
                    else:
                        ticker_name = api.finance_get_ticker_by_name(res[0])["result"]

                    if generated_query['metric'].lower().strip() == 'price':
                        response = api.finance_get_price_history(ticker_name)['result']
                    elif generated_query['metric'].lower().strip() == 'dividend':
                        response = api.finance_get_dividends_history(ticker_name)['result']
                    elif generated_query['metric'].lower().strip() == 'p/e ratio':
                        response = api.finance_get_pe_ratio(ticker_name)['result']
                    elif generated_query['metric'].lower().strip() == 'eps':
                        response = api.finance_get_eps(ticker_name)["result"]
                    elif generated_query['metric'].lower().strip() == 'marketcap' :
                        response = api.finance_get_market_capitalization(ticker_name)['result']
                    else:
                        response = api.finance_get_info(ticker_name)['result']
                        metric_value = get_metric_from_response(response, generated_query['metric'])
                        if metric_value is not None:
                            response = metric_value

                    try:
                        for formatted_time in formatted_time_list:
                            if formatted_time in response:
                                filtered_response = copy.deepcopy(response[formatted_time])
                            elif add_one_day(formatted_time) in response:
                                filtered_response = copy.deepcopy(response[add_one_day(formatted_time)])
                            elif subtract_one_day(formatted_time) in response:
                                filtered_response = copy.deepcopy(response[subtract_one_day(formatted_time)])
                            else:
                                filtered_response = copy.deepcopy(response)
                            kg_results.append({company_name + " " + generated_query["metric"]: filtered_response, 'time': formatted_time})
                    except:
                        kg_results.append({company_name + " " + generated_query["metric"]: response})

                except Exception as e:
                    print("Fail to parse the generated query")
                    pass

        kg_results = "<DOC>\n".join([str(res) for res in kg_results]) if len(kg_results) > 0 else ""
        return  kg_results

In [ ]:
### YOUR CODE HERE ###
### 다 볼 필요는 없고 위에 하나 정도만 대충 어떻게 되어 있는지 봐두면 될 듯. 패턴은 동일 함. ###
class CRAG(object):
    def __init__(self, server = None):
        self.server = os.environ.get('CRAG_SERVER', "http://10.2.0.165:8000")

    def finance_get_company_name(self, query:str):
        url = self.server + '/finance/get_company_name'
        headers={'accept': "application/json"}
        data = {'query': query}
        result = requests.post(url, json=data, headers=headers)
        return json.loads(result.text)

    def finance_get_ticker_by_name(self, query:str):
        url = self.server + '/finance/get_ticker_by_name'
        headers={'accept': "application/json"}
        data = {'query': query}
        result = requests.post(url, json=data, headers=headers)
        return json.loads(result.text)

    def finance_get_price_history(self, ticker_name:str):
        url = self.server + '/finance/get_price_history'
        headers={'accept': "application/json"}
        data = {'query': ticker_name}
        result = requests.post(url, json=data, headers=headers)
        return json.loads(result.text)

    def finance_get_detailed_price_history(self, ticker_name:str):
        url = self.server + '/finance/get_detailed_price_history'
        headers={'accept': "application/json"}
        data = {'query': ticker_name}
        result = requests.post(url, json=data, headers=headers)
        return json.loads(result.text)

    def finance_get_dividends_history(self, ticker_name:str):
        url = self.server + '/finance/get_dividends_history'
        headers={'accept': "application/json"}
        data = {'query': ticker_name}
        result = requests.post(url, json=data, headers=headers)
        return json.loads(result.text)

    def finance_get_market_capitalization(self, ticker_name:str):
        url = self.server + '/finance/get_market_capitalization'
        headers={'accept': "application/json"}
        data = {'query': ticker_name}
        result = requests.post(url, json=data, headers=headers)
        return json.loads(result.text)

    def finance_get_eps(self, ticker_name:str):
        url = self.server + '/finance/get_eps'
        headers={'accept': "application/json"}
        data = {'query': ticker_name}
        result = requests.post(url, json=data, headers=headers)
        return json.loads(result.text)

    def finance_get_pe_ratio(self, ticker_name:str):
        url = self.server + '/finance/get_pe_ratio'
        headers={'accept': "application/json"}
        data = {'query': ticker_name}
        result = requests.post(url, json=data, headers=headers)
        return json.loads(result.text)

    def finance_get_info(self, ticker_name:str):
        url = self.server + '/finance/get_info'
        headers={'accept': "application/json"}
        data = {'query': ticker_name}
        result = requests.post(url, json=data, headers=headers)
        return json.loads(result.text)

api = CRAG()

metric = "eps"

result = api.finance_get_company_name("microsoft")
pretty_json_print(result)
ticker_name = api.finance_get_ticker_by_name(result["result"][0])
pretty_json_print(ticker_name)

if metric == 'price':
    response = api.finance_get_price_history(ticker_name['result'])
elif metric == 'dividend':
    response = api.finance_get_dividends_history(ticker_name['result'])
elif metric == 'p/e ratio':
    response = api.finance_get_pe_ratio(ticker_name['result'])
elif metric == 'eps':
    response = api.finance_get_eps(ticker_name['result'])
elif metric == 'marketcap' :
    response = api.finance_get_market_capitalization(ticker_name['result'])
else:
    response = api.finance_get_info(ticker_name['result'])


In [ ]:
def generate_query(query):
    llm_input = prompt_generator(query)
    completion = oai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    temperature=0,
    messages=
    llm_input
    ).choices[0].message.content
    ##### 여기까지는 generate_response와 똑같다. 뒤에는 굳이 외워야 할까?

    try:
        completion = json.loads(completion)
    except:
        completion = extract_json_objects(completion)
    
    if "domain" in completion.keys():
        domain = completion["domain"]
        is_finance = domain == "finance"
    else:
        is_finance = False

    return completion, is_finance

# 아래 함수는 아마도? 안 나올 듯.......
def extract_json_objects(text, decoder=JSONDecoder()):
    """Find JSON objects in text, and yield the decoded JSON data
    """
    pos = 0
    results = []
    while True:
        match = text.find("{", pos)
        if match == -1:
            break
        try:
            result, index = decoder.raw_decode(text[match:])
            results.append(result)
            pos = match + index
        except ValueError:
            pos = match + 1
    return results

In [ ]:
## 이 knowledge graph가 나올까 싶긴 한데....
def get_finance_kg_results(generated_query):
    formatted_time_list = []
    if 'datetime' in generated_query:
        datetime_list = generated_query['datetime'].split(' - ')
        for datetime in datetime_list:
            formatted_time_list.append(convert_to_standard_format(datetime.strip()))


    kg_results = []
    res = ""
    if "market_identifier" in generated_query.keys() and generated_query["market_identifier"] is not None:
        if isinstance(generated_query["market_identifier"], str):
            company_names = generated_query["market_identifier"].split(",")
        else:
            company_names = generated_query["market_identifier"]

        for company_name in company_names:
            try:
                res = api.finance_get_company_name(company_name)["result"]

                if res == []:
                    ticker_name = company_name.upper()
                else:
                    ticker_name = api.finance_get_ticker_by_name(res[0])["result"]

                if generated_query['metric'].lower().strip() == 'price':
                    response = api.finance_get_price_history(ticker_name)['result']
                elif generated_query['metric'].lower().strip() == 'dividend':
                    response = api.finance_get_dividends_history(ticker_name)['result']
                elif generated_query['metric'].lower().strip() == 'p/e ratio':
                    response = api.finance_get_pe_ratio(ticker_name)['result']
                elif generated_query['metric'].lower().strip() == 'eps':
                    response = api.finance_get_eps(ticker_name)["result"]
                elif generated_query['metric'].lower().strip() == 'marketcap' :
                    response = api.finance_get_market_capitalization(ticker_name)['result']
                else:
                    response = api.finance_get_info(ticker_name)['result']
                    metric_value = get_metric_from_response(response, generated_query['metric'])
                    if metric_value is not None:
                        response = metric_value

                try:
                    for formatted_time in formatted_time_list:
                        if formatted_time in response:
                            filtered_response = copy.deepcopy(response[formatted_time])
                        elif add_one_day(formatted_time) in response:
                            filtered_response = copy.deepcopy(response[add_one_day(formatted_time)])
                        elif subtract_one_day(formatted_time) in response:
                            filtered_response = copy.deepcopy(response[subtract_one_day(formatted_time)])
                        else:
                            filtered_response = copy.deepcopy(response)
                        kg_results.append({company_name + " " + generated_query["metric"]: filtered_response, 'time': formatted_time})
                except:
                    kg_results.append({company_name + " " + generated_query["metric"]: response})

            except Exception as e:
                print("Fail to parse the generated query")
                pass

    kg_results = "<DOC>\n".join([str(res) for res in kg_results]) if len(kg_results) > 0 else ""
    return  kg_results